In [ ]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import wavfile

In [ ]:
from keras.layers import Conv1D, MaxPool1D, Activation, Dense, Input
import tensorflow as tf

# Prepare Training set

In [ ]:
annotations_path = '../data/MagnaTagATune/annotation_reduced.csv'
annotations = pd.read_csv(annotations_path, sep='\t')
train_set, _ = train_test_split(annotations['mp3_path'], train_size=0.005, test_size=0.001) 

In [ ]:
x_train = []
for value in tqdm(train_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_train.append(data)
x_train = np.array(x_train)[:,:,np.newaxis]
print("X train shape:",x_train.shape)

In [ ]:
y_train = annotations.loc[annotations['mp3_path'].isin(train_set)].drop(columns=['mp3_path','Unnamed: 0'])
y_train = y_train.values[:,np.newaxis,:]
print("Y train shape:",y_train.shape)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)

model = keras.Sequential()
model.add(Conv1D(filters=128, kernel_size=13, strides=13, input_shape=(x_train.shape[1],1)))
model.add(MaxPool1D(pool_size=13)) #
model.add(Conv1D(filters=256, kernel_size=7, strides=5))
model.add(MaxPool1D(pool_size=7))
model.add(Conv1D(filters=512, kernel_size=5, strides=3))
model.add(MaxPool1D(pool_size=5))
model.add(Conv1D(filters=512, kernel_size=3, strides=1))
model.add(MaxPool1D(pool_size=3))
model.add(Dense(units=y_train.shape[2], activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy')

In [ ]:
model.fit(x_train,y_train,epochs=1)